In [119]:
import re
# import openpyxl
# from openpyxl.styles import Alignment
from datetime import datetime
import pandas as pd

In [148]:
# 날짜 범위 생성
date_rng = pd.date_range(start='2024-02-19', end='2024-03-31', freq='D')

# 데이터프레임 생성
설정맵 = pd.DataFrame(columns=['박연수', '권세한', '권하은',
                           '박지은', '박하은', '손하진',
                           '송호석', '이신후', '이준수', '임혁규'], index=date_rng)

벌금맵 = pd.DataFrame(columns=['박연수', '권세한', '권하은',
                           '박지은', '박하은', '손하진',
                           '송호석', '이신후', '이준수', '임혁규'], index=date_rng)

In [149]:
file = pd.read_csv("6기.csv", encoding='UTF8')

pattern_set = r'\d{4}\s\w+\s설정' # 설정 정규식
pattern_success = r'\d{4}\s\w+\s성공' # 성공 정규식
pattern_fail = r'\d{4}\s\w+\s실패' # 실패 정규식
pattern_off = r'\d{4}\s\w+\soff' # 실패 정규식

filtered_rows = file[file['Message'].str.contains(f"{pattern_set}|{pattern_success}|{pattern_fail}", regex=True)]

In [154]:
def function1(data):
    pass
index = 76
DATE = filtered_rows['Date'][index]
USER = filtered_rows['User'][index]
MESSAGE = filtered_rows['Message'][index]

result_set = str(re.findall(pattern_set, MESSAGE))[2:][:-2]
result_off = str(re.findall(pattern_off, MESSAGE))[2:][:-2]
result_fail = str(re.findall(pattern_fail, MESSAGE))[2:][:-2]
result_success = str(re.findall(pattern_success, MESSAGE))[2:][:-2]

print(result_set)
print(result_off)
print(result_fail)
print(result_success)

result = [result_set, result_off, result_fail, result_success]

0222 손하진 설정





In [155]:
if USER in ['권세한', '임혁규', '이신후', '이준수']:
    DEADLINE = 21
else :
    DEADLINE = 10

date_str = '2024' + result_set[:4]
date_obj = datetime.strptime(date_str + f'{DEADLINE}', "%Y%m%d%H")

time_str = DATE
time_obj = datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S')

formatted_date = date_obj.strftime("%Y-%m-%d %H:%M:%S")

time_difference = time_obj - datetime.strptime(formatted_date, "%Y-%m-%d %H:%M:%S")

# Print the results in Korean
print("날짜:", date_obj.strftime("%Y년 %m월 %d일 %H시"))
print("date:", DATE)
print("시간 차이:", time_difference)

# 지각 논리 추가

날짜: 2024년 02월 22일 10시
date: 2024-02-22 11:06:48
시간 차이: 1:06:48


In [156]:
filtered_rows

,Date,User,Message
25,2024-02-19 20:57:16,손하진,0219 손하진 성공
26,2024-02-19 21:03:38,박연수,여러분 0220부터는 설정시 \n\n0220 박연수 설정\n- 이것\n- 저것\n\...
28,2024-02-19 21:12:00,박연수,0219 박연수 성공\n0220 박연수 설정\n- SSL 논문 2.5 까지 읽기\n...
30,2024-02-19 22:04:58,권세한,0220 권세한 설정\n- 수학의 샘 수학(하) 01. 원의 방정식 개념 정리 및 ...
33,2024-02-19 22:54:30,박연수,톡게시판 '공지': 여러분 0220부터는 설정시 \n\n0220 박연수 설정\n- ...
35,2024-02-20 00:35:25,송호석,0220 송호석 설정\n- 그림체 연구
36,2024-02-20 01:15:31,박하은,"0219 박하은 성공\n0219 박하은\n- rc 1,2 듣기\n- 알고리즘 1문제"
38,2024-02-20 01:17:41,박하은,"0220 박하은 설정\n- 전화 영어 \n- rc 3,4 듣기\n- 알고리즘 1문제"
39,2024-02-20 07:16:32,손하진,0220 손하진 설정\n- 선형대수 1-2 끝내기
42,2024-02-20 11:40:22,송호석,0220 송호석 성공\n0221 송호석 설정\n- 무궁화꽃 일러스트 완성\n- 그저...


In [157]:
# 결과 확인
date_str = date_obj.strftime("%Y-%m-%d %H:%M:%S")
print(date_str)

if time_difference.days < 0 or (time_difference.days == 0 and time_difference.seconds < 0):
    설정맵.loc[date_str[:10], USER] = 'O'

else : 
    설정맵.loc[date_str[:10], USER] = '지각'

print(date_str[:10])
print(설정맵)

2024-02-22 10:00:00
2024-02-22
            박연수  권세한  권하은  박지은  박하은  손하진  송호석  이신후  이준수  임혁규
2024-02-19  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
2024-02-20  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
2024-02-21  NaN  NaN  NaN  NaN  NaN  NaN    O  NaN  NaN  NaN
2024-02-22  NaN  NaN  NaN  NaN  NaN   지각  NaN  NaN  NaN  NaN
2024-02-23  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
2024-02-24  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
2024-02-25  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
2024-02-26  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
2024-02-27  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
2024-02-28  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
2024-02-29  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
2024-03-01  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
2024-03-02  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
2024-03-03  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
2024-03-04  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  Na